In [ ]:
import pandas as pd
import re, json

# Setup (public repo)

**Data files**
- Place input data files in `./data/` (you can exclude this folder from Git if the data isn't redistributable).
- Optionally add small, shareable examples in `./data_sample/`.

**OpenAI key**
- Set `OPENAI_API_KEY` as an environment variable (do not hard-code keys in notebooks).

```bash
export OPENAI_API_KEY="..."
```


In [ ]:
df = pd.read_csv('./data/indeed_data.csv')
print(len(df))
print(df.columns)
print(len(df['jobid'].unique()) == len(df))
print(df['salary_formatted'].isna().sum())
print(df['description_text'].iloc[0])

In [ ]:
df = df[[
    'jobid', 'company_name','job_title',
    'description_text', 'benefits', 'job_type',
    'location', 'salary_formatted'
]]

# Data Preprocessing

In [ ]:
import os
from openai import OpenAI

# Expect OPENAI_API_KEY to be set in your environment.
# For local dev, you can use a .env file and load it with python-dotenv (optional).
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
# # use 'location' to filter out listings that are not in the US
# # 1.5 minutes for 50 rows
# locations = list(df['location'].unique())
# print(len(locations))
# instructions = '''
#     You are an expert in determining whether a job listing is in the US.
#     Given the job location below, determine if the job is in the US.
#     Respond with only 'Yes' or 'No'.
# '''

# def get_in_us(location):
#     # check whether location has one of 50 states' abbreviations after a comma, followed by a space
#     if re.search(r',\s+(AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VA|WA|WV|WI|WY)\b', location):
#         return 'Yes'
#     else:
#         response = client.responses.create(
#             model="gpt-5-nano",
#         input=[
#             {"role": "developer", "content": instructions},
#             {"role": "user", "content": location},
#         ],
#         )
#         return response.output_text

# # df['in_us'] = df['location'].apply(get_in_us)
# for i in range(len(locations)):
#     location = locations[i]
#     in_us = get_in_us(location)
#     try:
#         df_in_us = pd.concat([df_in_us, pd.DataFrame({'location': [location], 'in_us': [in_us]})])
#     except:
#         df_in_us = pd.DataFrame({'location': [location], 'in_us': [in_us]})
#     if i % 100 == 0:
#         df_in_us.to_csv('Data Archive/filter_in_us_'+str(i)+'.csv', index=False)
# print(df_in_us['in_us'].value_counts())
# df_in_us.to_csv('Data Archive/filter_in_us.csv', index=False)

#df_in_us = pd.read_csv('Data Archive/filter_in_us.csv')
#AG notes: I mapped the file to new path on my dirve
df_in_us = pd.read_csv(r"./data/filter_in_us.csv")
df_in_us = df_in_us[df_in_us['in_us'] == 'Yes']
us_locations = list(df_in_us['location'].unique())
df['in_us'] = df['location'].apply(lambda x: 'Yes' if x in us_locations else 'No')
print(df['in_us'].value_counts())
df = df[df['in_us'] == 'Yes']
df.to_csv('indeed_data_in_us.csv', index=False)

In [ ]:
# # 2 minutes for 50 rows
# description_texts = list(df['description_text'].unique())
# print(len(description_texts))
# instructions = '''
#     You are an expert in categorizing job descriptions. Given the job description below, 
#     categorize the request into one of
#     "Administrator/Manager", "Aide/Assistant",
#     "Floaters/Substitutes", "Paraprofessional",
#     "Teacher/Lead Teacher", "Other staff responsible for children". 
#     Respond with only one of those words.
# '''

# def get_job_title(description):
#     response = client.responses.create(
#         model="gpt-5-nano",
#     input=[
#         {"role": "developer", "content": instructions},
#         {"role": "user", "content": description},
#     ],
#     )
#     return response.output_text

# # df['job_title2'] = df['description_text'].apply(get_job_title)
# for i in range(len(df)):
#     description = df['description_text'].iloc[i]
#     job_title = get_job_title(description)
#     try:
#         df_job_title = pd.concat([df_job_title, pd.DataFrame({'description_text': [description], 'job_title2': [job_title]})])
#     except:
#         df_job_title = pd.DataFrame({'description_text': [description], 'job_title2': [job_title]})
#     if i % 100 == 0:
#         df_job_title.to_csv('Data Archive/job_title2_'+str(i)+'.csv', index=False)
# print(df_job_title['job_title2'].value_counts())
# df_job_title.to_csv('Data Archive/job_title2.csv', index=False)

#df_job_title = pd.read_csv('Data Archive/job_title2.csv')
df_job_title = pd.read_csv(r"./data/job_title2.csv")
df_job_title = df_job_title.drop_duplicates(subset=['description_text'])
print(len(df_job_title))
df = pd.read_csv('indeed_data_in_us.csv')
print(len(df))
df = pd.merge(df, df_job_title, on='description_text', how='left')
print(len(df))
df.to_csv('indeed_data_in_us_with_job_title.csv', index=False)
print(df['job_title2'].value_counts())

In [ ]:
# # minimum qualifications (one by one)
# qualification_categories = [
#     '1. Teaching license, credential, or certificate',
#     '2. Educational degree or coursework',
#     '3. Curriculum development and lesson planning', 
#     '4. Supporting children with special needs or diverse learning needs',
#     '5. Classroom management and behavior guidance',
#     '6. Student assessment and progress monitoring',
#     '7. Communication skills (oral and written)',
#     '8. Parent and family engagement',
#     '9. Ensuring student safety and well-being', 
#     '10. Caregiving routines (feeding, diapering, toileting, etc.)',
#     '11. Use of technology in teaching or communication',
#     '12. Passion for working with young children / Child-centered mindset',
#     '13. Emotional qualities (patience, empathy, flexibility)',
#     '14. Collaboration and teamwork',
#     '15. Cultural competency and inclusivity',
#     '16. Willingness to learn / Professional development',
#     '17. Other (not listed above)'
# ]

# instructions = '''
#     You are an expert in analyzing job descriptions. Given the job description below, 
#     identify qualifications mentioned in the description and categorize them into one or more of the following categories:
#     {qualification_categories}
#     Respond only with the category numbers that best fit the qualification(s), separated by semicolons.
#     For example, if the qualification is a teaching license and a degree, respond with '1;2'.
#     If a qualification does not fit any of the categories, respond with '0'.
# '''

# def get_qualifications(description):
#     response = client.responses.create(
#         model="gpt-5-nano",
#     input=[
#         {"role": "developer", "content": instructions},
#         {"role": "user", "content": description},
#     ],
#     )
#     return response.output_text

# # df['qualification_categories'] = df['description_text'].apply(get_qualifications)
# for i in range(len(df)):
#     description = df['description_text'].iloc[i]
#     qual_categ = get_qualifications(description)
#     try:
#         df_qual_categ = pd.concat([df_qual_categ, pd.DataFrame({'description_text': [description], 'qual_categ': [qual_categ]})])
#     except:
#         df_qual_categ = pd.DataFrame({'description_text': [description], 'qual_categ': [qual_categ]})
#     if i % 100 == 0:
#         df_qual_categ.to_csv('Data Archive/qual_categ'+str(i)+'.csv', index=False)
# print(df_qual_categ['qual_categ'].value_counts())
# df_qual_categ.to_csv('Data Archive/qual_categ_FINAL.csv', index=False)

In [ ]:
# # Qualification categories (batch)
# qualification_categories = [
#     '1. Teaching license, credential, or certificate',
#     '2. Educational degree or coursework',
#     '3. Curriculum development and lesson planning', 
#     '4. Supporting children with special needs or diverse learning needs',
#     '5. Classroom management and behavior guidance',
#     '6. Student assessment and progress monitoring',
#     '7. Communication skills (oral and written)',
#     '8. Parent and family engagement',
#     '9. Ensuring student safety and well-being', 
#     '10. Caregiving routines (feeding, diapering, toileting, etc.)',
#     '11. Use of technology in teaching or communication',
#     '12. Passion for working with young children / Child-centered mindset',
#     '13. Emotional qualities (patience, empathy, flexibility)',
#     '14. Collaboration and teamwork',
#     '15. Cultural competency and inclusivity',
#     '16. Willingness to learn / Professional development',
#     '17. Other (not listed above)'
# ]

# instructions = f'''
#     You are an expert in analyzing job descriptions. 
#     Given multiple job descriptions, identify qualifications mentioned in each one and categorize them into one or more of the following **exact categories**.d
#     Each category has a fixed number:
#     {qualification_categories}

#     Rules:
#     - Use ONLY the numbers shown above (1 to 17).
#     - Respond with exactly one line per description, in the same order as provided.
#     - Each line must contain only the category numbers, separated by semicolons (e.g., "2;5;7").
#     - If none apply, respond with "0".
#     - Do not ask for clarification, do not explain, do not add extra text.
# '''


# def get_qualifications_batch(descriptions):
#     input_text = "\n\n---\n\n".join(
#         [f"Description {i+1}:\n{desc}" for i, desc in enumerate(descriptions)]
#     )
#     response = client.responses.create(
#         model="gpt-5-nano",
#         input=[
#             {"role": "developer", "content": instructions},
#             {"role": "user", "content": input_text},
#         ],
#     )
    
#     return response.output_text.strip().split("\n")

# # Process DataFrame in batches
# batch_size = 10
# for i in range(600, len(df), batch_size):
#     batch = df['description_text'].iloc[i:i+batch_size].tolist()
#     batch_results = get_qualifications_batch(batch)
#     if len(batch_results) == len(batch):
#         temp = pd.DataFrame({'description_text': batch, 'qual_categ': batch_results})
#     else:
#         print('Error at i = ', i, ': batch_results length does not match batch length')
#         print(batch_results)
#     try:
#         df_qual_categ = pd.concat([df_qual_categ, temp])
#     except:
#         df_qual_categ = temp
#     # Save checkpoint every 50 batches
#     if i % (batch_size * 50) == 0:
#         df_qual_categ.to_csv(f"Data Archive/qual_categ_{i}.csv", index=False)

# df_qual_categ.to_csv("Data Archive/qual_categ_FINAL.csv", index=False)
# print(df_qual_categ['qual_categ'].value_counts())

In [ ]:
# tabulate number of job postings in each category
#df_qual_categ = pd.read_csv('Data Archive/qual_categ_FINAL.csv').query('qual_categ.notna()').drop_duplicates(subset=['description_text'])
df_qual_categ = pd.read_csv(
    r"./data/qual_categ_FINAL.csv"
).query('qual_categ.notna()').drop_duplicates(subset=['description_text'])

df = pd.merge(df, df_qual_categ, on='description_text', how='left')
N = len(df.query('qual_categ.notna()'))
for i in range(1, 18):
    df_qual_categ['temp'] = df_qual_categ['qual_categ'].apply(lambda x: str(i) in x.split(';'))
    print(i, df_qual_categ['temp'].sum(), round(df_qual_categ['temp'].sum() / N, 4))

In [ ]:
# salary
temp = df.query('salary_formatted.notna()').copy()
print(len(temp))
print(len(temp.query('salary_formatted.str.contains("hour")')))
print(len(temp.query('salary_formatted.str.contains("day")')))
print(len(temp.query('salary_formatted.str.contains("month")')))
print(len(temp.query('salary_formatted.str.contains("year")')))

In [ ]:
temp2 = temp.query('salary_formatted.str.contains("hour")').copy()
# extract first number after $ sign, doesn't have to include decimal point
temp2['hourly_rate'] = temp2['salary_formatted'].str.extract(r'\$(\d+)').astype(float)
print(temp2[['salary_formatted', 'hourly_rate']].head())
print(temp2['hourly_rate'].describe())

In [ ]:
# benefits
# combine 401 and 403 related benefits into 'Retirement plan'
temp3 = df.query('benefits.notna()').copy()
temp3['benefits'] = temp3['benefits'].apply(lambda x: 'Retirement plan' if '401' in x or '403' in x or '457' in x else x)
all_benefits = [benefit for sublist in temp3['benefits'] for benefit in sublist.replace('[', '').replace(']', '').replace('\"', '').split(',')]
unique_benefits = sorted(set(all_benefits))
print(unique_benefits)

# tabulate number of job postings in each benefit category
ns = []
for benefit in unique_benefits:
    n = temp3['benefits'].apply(lambda x: benefit in x).sum()
    ns.append(n)
# sort benefits by number of job postings
sorted_benefits = [[ns, x] for ns, x in sorted(zip(ns, unique_benefits), key=lambda pair: pair[0], reverse=True)]
print(sorted_benefits)
sorted_benefits = [x for ns, x in sorted(zip(ns, unique_benefits), key=lambda pair: pair[0], reverse=True)]
print(sorted_benefits)


In [ ]:
temp3['benefits'].iloc[0]

### Next steps
- generate variables for job locations: state (e.g., CA, MA, DC), urban/rural (dichotomous variable)
- extract more salary data from 'description_text' if the salary data is missing in the 'salary_formatted' column using GPT.
- extract more benefits data from 'description_text' if the benefits data is missing in the 'benefits' column using GPT.
- run tabulation or descriptive statistics of hourly salary and benefits by state, urban/rural, and job_title2.


### LOCATION 
- generate variables for job locations: state (e.g., CA, MA, DC), urban/rural (dichotomous variable)

In [ ]:
#LOCATION - - generate variables for job locations: state (e.g., CA, MA, DC), urban/rural (dichotomous variable)


import re
import pandas as pd

def parse_location(loc):
    """
    Parse a free-text job location into city, state, and zip.
    Handles US states, DC, and territories (PR, GU, VI, AS, MP).
    Cleans up cases where street addresses were mistaken as cities.
    """
    if pd.isna(loc):
        return pd.Series({"city": None, "state": None, "zip": None})
    s = str(loc).strip()

    # ---- Handle generic "United States"/Remote ----
    if s.lower() in {"united states", "usa", "u.s.a.", "us", "remote"}:
        return pd.Series({"city": None, "state": None, "zip": None})

    # ---- Case 1: Address + City, ST ZIP ----
    m = re.match(r'^(.*?),\s*([^,]+),\s*([A-Z]{2})\s+(\d{5})(?:-\d{4})?$',
                 s, flags=re.IGNORECASE)
    if m:
        return pd.Series({
            "city": m.group(2).strip().title(),       # last token before state
            "state": m.group(3).upper(),
            "zip": m.group(4).zfill(5)                # preserve leading zeros
        })

    # ---- Case 2: City, ST ZIP (no street) ----
    m = re.match(r'^(.*?),\s*([A-Z]{2})\s+(\d{5})(?:-\d{4})?$',
                 s, flags=re.IGNORECASE)
    if m:
        return pd.Series({
            "city": m.group(1).strip().title(),
            "state": m.group(2).upper(),
            "zip": m.group(3).zfill(5)
        })

    # ---- Case 3: Address + City, ST (no ZIP) ----
    m = re.match(r'^(.*?),\s*([^,]+),\s*([A-Z]{2})$', s, flags=re.IGNORECASE)
    if m:
        return pd.Series({
            "city": m.group(2).strip().title(),
            "state": m.group(3).upper(),
            "zip": None
        })

    # ---- Case 4: City, ST (no ZIP, no street) ----
    m = re.match(r'^(.*?),\s*([A-Z]{2})$', s, flags=re.IGNORECASE)
    if m:
        return pd.Series({
            "city": m.group(1).strip().title(),
            "state": m.group(2).upper(),
            "zip": None
        })

    # ---- Case 5: State only (full names + territories) ----
    state_names = {
        "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA",
        "Colorado":"CO","Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA",
        "Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS",
        "Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA",
        "Michigan":"MI","Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT",
        "Nebraska":"NE","Nevada":"NV","New Hampshire":"NH","New Jersey":"NJ","New Mexico":"NM",
        "New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK",
        "Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC",
        "South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT",
        "Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY",
        "District Of Columbia":"DC",
        "Puerto Rico":"PR","Guam":"GU","Virgin Islands":"VI",
        "American Samoa":"AS","Northern Mariana Islands":"MP"
    }
    s_clean = s.replace(" State","").title().strip()
    if s_clean in state_names:
        return pd.Series({"city": None, "state": state_names[s_clean], "zip": None})

    # ---- Fallback ----
    return pd.Series({"city": None, "state": None, "zip": None})


# --- Apply parser ---
df[['city','state','zip']] = df['location'].apply(parse_location)


In [ ]:
# --- Data Quality Checks on location ---

import pandas as pd

print("Total rows:", len(df))
print("Unique states found:", df['state'].nunique())
print("Unique zips found:", df['zip'].nunique())
print("="*50)

# Define masks
problem_mask = df['city'].isna() & df['state'].isna() & df['zip'].isna()
mask_city_no_state = df['city'].notna() & df['state'].isna()
mask_state_no_city = df['state'].notna() & df['city'].isna()
mask_no_zip = df['zip'].isna() & (df['city'].notna() | df['state'].notna())
mask_city_addresslike = df['city'].notna() & df['city'].str.contains(r'\d', na=False)
mask_county = df['city'].notna() & df['city'].str.contains("county", case=False, na=False)
mask_pr = df['state'] == "PR"

# 1. Problematic rows
print("No city, no state, no zip:", problem_mask.sum())
print(df.loc[problem_mask, ['location','city','state','zip']].head(20))
print("="*50)

# 2. City but missing state
print("City present but missing state:", mask_city_no_state.sum())
print(df.loc[mask_city_no_state, ['location','city','state','zip']].head(20))
print("="*50)

# 3. State but missing city
print("State present but missing city:", mask_state_no_city.sum())
print(df.loc[mask_state_no_city, ['location','city','state','zip']].head(20))
print("="*50)

# 4. No zip but city/state present
print("No zip but has city/state:", mask_no_zip.sum())
print(df.loc[mask_no_zip, ['location','city','state','zip']].head(20))
print("="*50)

# 5. City looks like address
print("City contains digits (likely address):", mask_city_addresslike.sum())
print(df.loc[mask_city_addresslike, ['location','city','state','zip']].head(20))
print("="*50)

# 6. City includes 'county'
print("City includes 'county':", mask_county.sum())
print(df.loc[mask_county, ['location','city','state','zip']].head(20))
print("="*50)

# 7. Puerto Rico
print("Rows with PR state:", mask_pr.sum())
print(df.loc[mask_pr, ['location','city','state','zip']].head(20))
print("="*50)

# --- Summary Table ---
summary = pd.DataFrame({
    "Check": [
        "No city/state/zip",
        "City but no state",
        "State but no city",
        "No zip but city/state present",
        "City looks like address (digits)",
        "City includes 'county'",
        "Puerto Rico rows"
    ],
    "Count": [
        problem_mask.sum(),
        mask_city_no_state.sum(),
        mask_state_no_city.sum(),
        mask_no_zip.sum(),
        mask_city_addresslike.sum(),
        mask_county.sum(),
        mask_pr.sum()
    ]
})

print("\n=== Summary of Data Quality Checks ===")
print(summary)


In [ ]:
#adding census ubran/rurality

import numpy as np

# Load RUCA reference file
census_ref = pd.read_csv("RUCA-codes-2020-zipcode.csv")

census_ref = census_ref.rename(columns={
    "ZIPCode": "zip",
    "POName": "city",
    "State": "state",
    "PrimaryRUCA": "ruca_code"
})

# standardize
census_ref['zip']   = census_ref['zip'].astype(str).str.zfill(5)
census_ref['city']  = census_ref['city'].astype(str).str.title().str.strip()
census_ref['state'] = census_ref['state'].astype(str).str.upper().str.strip()

# urban/rural flag
census_ref['ruca_code'] = pd.to_numeric(census_ref['ruca_code'], errors='coerce')
census_ref['urban_rural'] = census_ref['ruca_code'].apply(lambda x: "Urban" if x < 4 else "Rural")

# ---------- STEP 1: ZIP merge ----------
df_zip_merge = df.merge(
    census_ref[['zip','urban_rural','ruca_code']],
    on='zip', how='left', indicator=True
)

zip_matches = df_zip_merge[df_zip_merge['_merge'] == 'both'].drop(columns=['_merge'])
unmatched   = df_zip_merge[df_zip_merge['_merge'] == 'left_only'] \
                .drop(columns=['urban_rural','ruca_code','_merge'])

print("ZIP matches:", len(zip_matches))
print("Unmatched after ZIP:", len(unmatched))


# ---------- STEP 2: Collapse census to city+state ----------
census_ref['urban_flag'] = (census_ref['ruca_code'] < 4).astype(int)

city_level = (
    census_ref.dropna(subset=['city','state'])
              .groupby(['city','state'], as_index=False)
              .agg(
                  n_zips=('zip','nunique'),
                  urban_share=('urban_flag','mean')
              )
)
city_level['urban_rural'] = np.where(city_level['urban_share'] >= 0.5, 'Urban', 'Rural')

# ---------- STEP 3: City+State merge on only the unmatched ----------
df_city_merge = unmatched.merge(
    city_level[['city','state','urban_rural']],
    on=['city','state'], how='left', indicator=True
)

city_matches   = df_city_merge[df_city_merge['_merge'] == 'both'].drop(columns=['_merge'])
still_unmatched = df_city_merge[df_city_merge['_merge'] == 'left_only']

print("City+State matches:", len(city_matches))
print("Still unmatched:", len(still_unmatched))


# ---------- STEP 4: Combine ----------
df_final = pd.concat([zip_matches, city_matches], ignore_index=True)

print("Final merged dataset:", len(df_final))
print("Unmatched jobs overall:", len(still_unmatched))



In [ ]:
#checking those that did not merge
import pandas as pd

# Load unmatched file
unmatched = pd.read_csv("unmatched_locations.csv")

print("Starting unmatched:", len(unmatched))

# 1. Drop "United States" / USA / US
mask_us = unmatched['location'].str.strip().str.lower().isin(["united states", "usa", "u.s.a.", "us"])
print("Drop 'United States':", mask_us.sum())
unmatched = unmatched.loc[~mask_us].copy()

# 2. Drop state-only
mask_state_only = unmatched['city'].isna() & unmatched['state'].notna() & unmatched['zip'].isna()
print("Drop state-only:", mask_state_only.sum())
unmatched = unmatched.loc[~mask_state_only].copy()

# 3. Drop county values
mask_county = unmatched['city'].notna() & unmatched['city'].str.contains("county", case=False, na=False)
print("Drop county:", mask_county.sum())
unmatched = unmatched.loc[~mask_county].copy()

# 4. Drop single-word only (e.g., "Lewes")
mask_single_word_city = unmatched['location'].str.strip().str.count(r'\s+') == 0
print("Drop single-word city:", mask_single_word_city.sum())
unmatched = unmatched.loc[~mask_single_word_city].copy()

print("Remaining after cleanup:", len(unmatched))

# Save cleaned file
unmatched.to_csv("unmatched_locations_cleaned.csv", index=False)


### SALARY
- extract more salary data from 'description_text' if the salary data is missing in the 'salary_formatted' column using GPT.

In [ ]:
#SALARY - extract more salary data from 'description_text' if the salary data is missing in the 'salary_formatted' column using GPT.

missing_salary_mask = df['salary_formatted'].isna() | (df['salary_formatted'].str.strip() == "")
df_missing_salary = df.loc[missing_salary_mask, ['jobid','description_text']].copy()

print("Jobs with missing salary:", len(df_missing_salary))
print(df_missing_salary.head(3))


In [ ]:

# ---------------------------
# EXTRACTING SALARY DATA
# ---------------------------
#import re, json
#import pandas as pd

# ---------------------------
# 1. Regex function (safe on all rows)
# ---------------------------
#def regex_salary(text):
#    if pd.isna(text):
#        return None
#    m = re.findall(
#        r"\$\s?\d[\d,]*(?:\.\d{1,2})?(?:\s*-\s*\$?\d[\d,]*(?:\.\d{1,2})?)?",
#        str(text),
#        flags=re.IGNORECASE
#    )
#    return m[0] if m else None

# Apply regex across all rows
#df_missing_salary['regex_salary'] = df_missing_salary['description_text'].apply(regex_salary)

#print("Regex captured:", df_missing_salary['regex_salary'].notna().sum())
#print("Still missing after regex:", df_missing_salary['regex_salary'].isna().sum())


# ---------------------------
# 2. GPT extraction function (sample only)
# ---------------------------
#def extract_salary_from_text(text, jobid=None):
#    prompt = f"""
#    Extract the **base pay information only** from this job posting.
#    - Ignore bonuses, stipends, or one-time benefits unless no base salary is given.
#    - Look for pay written as $, 'per hour', 'per year', 'monthly', or 'daily'.
#    - If multiple are listed (e.g., bonus + hourly), keep only the base hourly/yearly/monthly salary.
#    - If no salary is explicitly mentioned, return exactly "None".

#    Text:
#   {text}

 #   Respond strictly as JSON in this format:
 #   {{
 #      "salary_text": "<raw salary phrase or None>",
 #       "type": "hourly/daily/monthly/annual/None",
 #       "min": <number or null>,
 #       "max": <number or null>,
 #       "currency": "USD"
 #   }}
 #   """
 #   try:
 #       response = client.chat.completions.create(
 #           model="gpt-4o-mini",
 #           messages=[{"role": "user", "content": prompt}],
 #           temperature=0
 #       )
#        return response.choices[0].message.content
#    except Exception:
#        return None


# ---------------------------
# 3. Clean GPT JSON output
# ---------------------------
#def clean_gpt_output(raw):
#    if pd.isna(raw) or raw is None:
#        return None
#    cleaned = re.sub(r"^```json|```$", "", str(raw), flags=re.MULTILINE).strip()
#    try:
#        return json.loads(cleaned)
#    except:
#        return None

# ---------------------------
# 4. Run GPT on a SAMPLE of 100 rows
# ---------------------------
##sample_rows = df_missing_salary.head(100)   # <---- sampel of only 100 rows to start
#sample_rows = df_missing_salary   # <--  ALL rows missing salary


# gpt_results = []
# for idx, row in sample_rows.iterrows():
#    result = extract_salary_from_text(row['description_text'], jobid=row['jobid'])
#    gpt_results.append({"jobid": row['jobid'], "salary_extracted": result})

# Collect into DataFrame
# df_gpt = pd.DataFrame(gpt_results)
# df_gpt['parsed'] = df_gpt['salary_extracted'].apply(clean_gpt_output)


# ---------------------------
# 5. Merge back for sample
# ---------------------------
# df_debug = sample_rows.merge(df_gpt[['jobid','parsed']], on="jobid", how="left")

# Final structured salary fields
# df_debug['final_salary_text'] = df_debug.apply(
#    lambda r: r['regex_salary'] if pd.notna(r['regex_salary'])
#              else (r['parsed'].get('salary_text') if isinstance(r['parsed'], dict) else None),
#    axis=1
#)
#df_debug['final_type'] = df_debug['parsed'].apply(lambda x: x.get('type') if isinstance(x, dict) else None)
#df_debug['final_min']  = df_debug['parsed'].apply(lambda x: x.get('min') if isinstance(x, dict) else None)
#df_debug['final_max']  = df_debug['parsed'].apply(lambda x: x.get('max') if isinstance(x, dict) else None)

# ---------------------------
# 6. Monitoring capture
# ---------------------------
#print("Total rows in df_missing_salary:", len(df_missing_salary))
#print("Regex captured:", df_missing_salary['regex_salary'].notna().sum())
#print("Still missing after regex:", df_missing_salary['regex_salary'].isna().sum())

#print("GPT sample size:", len(sample_rows))
#print("GPT parsed valid:", df_gpt['parsed'].notna().sum())

# Count how many GPT acually extracted a salary
#gpt_with_salary = df_gpt['parsed'].apply(
 #   lambda x: isinstance(x, dict) and x.get('salary_text') not in [None, "", "None"]
#).sum()

#print("GPT extracted salary:", gpt_with_salary)

# Also, how many still missing after GPT
#missing_after_gpt = len(sample_rows) - gpt_with_salary
#print("Missing after GPT:", missing_after_gpt)

# Show a peek at structured results
#print(df_debug[['jobid','regex_salary','final_salary_text','final_type','final_min','final_max']].head(50))

# Save the full missing-salary dataframe
#df_debug.to_csv("salary_extraction_complete.csv", index=False)




In [ ]:
#CLEANING UP SALARY DATA BY STANDARDIZING TO HOURS
import pandas as pd

# Load previously saved GPT+regex results
df_debug = pd.read_csv("salary_extraction_complete.csv")

print("Rows loaded:", len(df_debug))
print(df_debug.head(5))

df_debug['final_min'] = df_debug['final_min'].fillna(df_debug['final_max'])
df_debug['final_max'] = df_debug['final_max'].fillna(df_debug['final_min'])

def convert_to_hourly(row):
    if pd.isna(row['final_min']) and pd.isna(row['final_max']):
        return pd.Series([None, None])
    
    salary_type = str(row['final_type']).lower()
    min_val = row['final_min']
    max_val = row['final_max']

    if salary_type == "hourly":
        return pd.Series([min_val, max_val])
    elif salary_type == "daily":
        return pd.Series([min_val/8 if min_val else None,
                          max_val/8 if max_val else None])
    elif salary_type == "monthly":
        return pd.Series([min_val/(20*8) if min_val else None,
                          max_val/(20*8) if max_val else None])
    elif salary_type == "annual":
        return pd.Series([min_val/(250*8) if min_val else None,
                          max_val/(250*8) if max_val else None])
    else:
        return pd.Series([None, None])

df_debug[['hourly_min','hourly_max']] = df_debug.apply(convert_to_hourly, axis=1)

df_debug.to_csv("salary_extraction_hourly.csv", index=False)

print(df_debug[['hourly_min','hourly_max']].describe())




In [ ]:
#strandarizing salary  info in main dataset 
import re
import pandas as pd

# ---------------------------
# 1. Parse salary_formatted into min/max + type
# ---------------------------
def parse_salary_range(text):
    """
    Parse salary_formatted values like:
      "$13.00 - $13.50 an hour"
      "$45,000 - $50,000 a year"
      "$2,500 a month"
      "$150 per day"
    Returns dict with min, max, type.
    """
    if pd.isna(text) or str(text).strip() == "":
        return {"min": None, "max": None, "type": None}

    s = str(text).lower().strip()

    # Detect type
    if "hour" in s:
        pay_type = "hourly"
    elif "year" in s or "annum" in s:
        pay_type = "annual"
    elif "month" in s:
        pay_type = "monthly"
    elif "day" in s:
        pay_type = "daily"
    elif "week" in s:
        pay_type = "weekly"
    else:
        pay_type = None

    # Extract numbers
    nums = re.findall(r"\$?\s?([\d,]+(?:\.\d{1,2})?)", s)
    nums = [float(x.replace(",", "")) for x in nums]

    if len(nums) == 1:
        min_val = max_val = nums[0]
    elif len(nums) >= 2:
        min_val, max_val = nums[0], nums[1]
    else:
        min_val = max_val = None

    return {"min": min_val, "max": max_val, "type": pay_type}


# Apply parser to main df
parsed = df['salary_formatted'].apply(parse_salary_range)

df['salary_min'] = parsed.apply(lambda x: x['min'])
df['salary_max'] = parsed.apply(lambda x: x['max'])
df['salary_type'] = parsed.apply(lambda x: x['type'])

# ---------------------------
# 2. Convert everything to hourly
# ---------------------------
def to_hourly(row):
    if pd.isna(row['salary_min']) or pd.isna(row['salary_max']):
        return (None, None)

    min_val, max_val, pay_type = row['salary_min'], row['salary_max'], row['salary_type']

    if pay_type == "hourly":
        return (min_val, max_val)
    elif pay_type == "daily":
        return (min_val/8, max_val/8)   # assume 8 hours/day
    elif pay_type == "weekly":
        return (min_val/40, max_val/40) # assume 40 hrs/week
    elif pay_type == "monthly":
        return (min_val/(20*8), max_val/(20*8)) # 20 workdays * 8 hrs
    elif pay_type == "annual":
        return (min_val/2000, max_val/2000) # assume 2000 hrs/year
    else:
        return (None, None)

df[['hourly_min','hourly_max']] = df.apply(to_hourly, axis=1, result_type="expand")

# ---------------------------
# 3. Sanity check
# ---------------------------
print(df[['jobid','salary_formatted','salary_type','salary_min','salary_max','hourly_min','hourly_max']].head(20))





In [ ]:
# ---------------------------
# Salary coverage counts BEFORE merge
# ---------------------------
print("=== Coverage BEFORE merge ===")
print("Total rows in df (main):", len(df))
print("Rows in df with salary_formatted parsed:", df['hourly_min'].notna().sum())
print("Total rows in df_debug (GPT/regex supplement):", len(df_debug))
print("Rows in df_debug with hourly salary info:", df_debug['hourly_min'].notna().sum())

# ---------------------------
# Merge salaries + reattach urban/rural here
# ---------------------------
df_merged = (
    df.merge(
        df_debug[['jobid','final_salary_text','final_type','final_min','final_max','hourly_min','hourly_max']],
        on="jobid", how="left", suffixes=("", "_gpt")
    )
    # add urban/rural back in from df_final (location step)
    .merge(
        df_final[['jobid','urban_rural']], 
        on="jobid", how="left"
    )
)

# Prefer GPT/regex when available
df_merged['salary_text_final'] = df_merged['final_salary_text'].combine_first(df_merged['salary_formatted'])
df_merged['salary_type_final'] = df_merged['final_type'].combine_first(df_merged['salary_type'])
df_merged['salary_min_final']  = df_merged['final_min'].combine_first(df_merged['salary_min'])
df_merged['salary_max_final']  = df_merged['final_max'].combine_first(df_merged['salary_max'])
df_merged['hourly_min_final']  = df_merged['hourly_min_gpt'].combine_first(df_merged['hourly_min'])
df_merged['hourly_max_final']  = df_merged['hourly_max_gpt'].combine_first(df_merged['hourly_max'])

# ---------------------------
# Salary coverage counts AFTER merge
# ---------------------------
print("\n=== Coverage AFTER merge ===")
print("Total rows in df_merged:", len(df_merged))
print("Rows in df_merged with hourly salary info:",
      df_merged['hourly_min_final'].notna().sum())

print("\nUrban/Rural distribution after merge:")
print(df_merged['urban_rural'].value_counts(dropna=False))


In [ ]:
import pandas as pd

# -----------------------------
# Outlier thresholds
# -----------------------------
min_threshold = 2      # implausibly low (< $2/hr)
max_threshold = 400    # implausibly high (> $400/hr)

# -----------------------------
# Outlier detection on merged dataset
# -----------------------------
outlier_mask = (
    (df_merged['hourly_min_final'] < min_threshold) |
    (df_merged['hourly_max_final'] > max_threshold) |
    (df_merged['hourly_min_final'] > df_merged['hourly_max_final'])
)

df_outliers = df_merged.loc[outlier_mask].copy()
df_cleaned = df_merged.loc[~outlier_mask].copy()

# -----------------------------
# Diagnostics
# -----------------------------
print("=== Outlier Detection ===")
print("Outliers detected:", len(df_outliers))
print("Original dataset size:", len(df_merged))
print("Cleaned dataset size:", len(df_cleaned))
print("Rows removed:", len(df_merged) - len(df_cleaned))

print("\nSample outliers:")
print(df_outliers[['jobid','salary_text_final','salary_type_final',
                   'salary_min_final','salary_max_final',
                   'hourly_min_final','hourly_max_final']].head(20))

# -----------------------------
# Save cleaned dataset
# -----------------------------
df_cleaned.to_csv("job_data_cleaned_no_outliers.csv", index=False)


### Descriptives

In [ ]:
# -----------------------------
# Descriptive statistics by urban/rural (CLEANED DATA)
# -----------------------------
df_desc = df_cleaned[df_cleaned['hourly_min_final'].notna()]

summary = df_desc.groupby('urban_rural').agg(
    n_jobs=('jobid', 'count'),
    mean_hourly_min=('hourly_min_final','mean'),
    mean_hourly_max=('hourly_max_final','mean'),
    median_hourly_min=('hourly_min_final','median'),
    median_hourly_max=('hourly_max_final','median'),
    min_hourly=('hourly_min_final','min'),
    max_hourly=('hourly_max_final','max')
).reset_index()

print("\n=== Hourly Salary Descriptives by Urban/Rural (Cleaned) ===")
print(summary)

overall = df_desc[['hourly_min_final','hourly_max_final']].describe()
print("\n=== Overall Hourly Salary Descriptives (Cleaned) ===")
print(overall)


In [ ]:
print(df_cleaned.columns)


In [ ]:
import pandas as pd

# -----------------------------
# Descriptive statistics by STATE
# -----------------------------
# Filter to rows that have salary + state info
df_state_desc = df_cleaned[
    df_cleaned['hourly_min_final'].notna() & df_cleaned['state'].notna()
]

# Group by state and summarize
state_summary = df_state_desc.groupby('state').agg(
    n_jobs=('jobid', 'count'),
    mean_hourly_min=('hourly_min_final','mean'),
    mean_hourly_max=('hourly_max_final','mean'),
    median_hourly_min=('hourly_min_final','median'),
    median_hourly_max=('hourly_max_final','median'),
    min_hourly=('hourly_min_final','min'),
    max_hourly=('hourly_max_final','max')
).reset_index()

# Sort by median to see highest/lowest states
state_summary = state_summary.sort_values('median_hourly_min', ascending=False)

print("\n=== Hourly Salary Descriptives by State (Cleaned) ===")
print(state_summary.head(20))   # top 20 states
print("\nLowest 10 states by median hourly_min:")
print(state_summary.tail(10))


## creating data table
-In the table, the top row includes national median, and the next 50 rows present state-level medians. 
-Columns 1-2 are min & max of hourly wage. 
-Column 3 is state minimum wages
-column 4-5 is BLS data for preschool and K teachers

In [ ]:
import pandas as pd
import re

# -----------------------------
# 1. Load BLS OES Excel files (rows 6–58 only, no national row)
# -----------------------------
kindergarten = pd.read_excel(
    "OES_Kindergarten_Teacher.xlsx",
    skiprows=5,  # skip first 5 rows so A6 is first
    nrows=53     # rows 6–58 = 53 rows
)

preschool = pd.read_excel(
    "OES_Preschool_Teacher.xlsx",
    skiprows=5,
    nrows=55
)

# -----------------------------
# 2. Extract & clean "state" column
# -----------------------------
def clean_state(name):
    """Remove the (01-00000) codes from 'Area Name' and keep state name only."""
    return re.sub(r"\s*\(.*?\)", "", str(name)).strip()

kindergarten["state"] = kindergarten["Area Name"].apply(clean_state)
preschool["state"] = preschool["Area Name"].apply(clean_state)

# -----------------------------
# 3. Select Annual Median (Column H → index 7)
# -----------------------------
kindergarten["annual_median"] = kindergarten.iloc[:, 7]
preschool["annual_median"] = preschool.iloc[:, 7]

# -----------------------------
# 4. Convert to hourly using 2,080 hours/year
# -----------------------------
kindergarten["hourly_median_kindergarten"] = kindergarten["annual_median"] / 2080
preschool["hourly_median_preschool"] = preschool["annual_median"] / 2080

# -----------------------------
# 5. Keep only needed columns
# -----------------------------
kindergarten = kindergarten[["state", "hourly_median_kindergarten"]]
preschool = preschool[["state", "hourly_median_preschool"]]

# -----------------------------
# 6. Merge Preschool & Kindergarten datasets
# -----------------------------
bls_wages = preschool.merge(kindergarten, on="state", how="outer")

# -----------------------------
# 7. Quick sanity check
# -----------------------------
print("BLS hourly wage dataset shape:", bls_wages.shape)
print(bls_wages.head(10))

# -----------------------------
# 8. Save to CSV for future use
# -----------------------------
bls_wages.to_csv("BLS_hourly_preschool_kindergarten.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

# File path
file_path = r"./data/state min wage.xlsx"

# Load file
min_wage = pd.read_excel(file_path, usecols=[0, 1])

# Rename columns
min_wage.columns = ["state_name", "state_min_wage_raw"]

def clean_wage(val):
    """Clean wage strings into a numeric float."""
    if pd.isna(val):
        return np.nan
    
    s = str(val).strip()
    
    # Replace $ and commas
    s = s.replace("$", "").replace(",", "")
    
    # Handle 'Varies'
    if s.lower() == "varies":
        return np.nan
    
    # Handle ranges like '15.50/16.50' or '15.50-16.50'
    if "/" in s or "-" in s:
        parts = re.split(r'[-/]', s)
        try:
            nums = [float(p) for p in parts if p.strip() != ""]
            return np.mean(nums) if nums else np.nan
        except:
            return np.nan
    
    # Otherwise just convert to float
    try:
        return float(s)
    except:
        return np.nan

# Apply cleaner
min_wage["state_min_wage"] = min_wage["state_min_wage_raw"].apply(clean_wage)

# Drop raw if not needed
min_wage = min_wage.drop(columns=["state_min_wage_raw"])

print(min_wage.head(15))



In [ ]:
import pandas as pd

# --- state mapping (long -> abbrev) ---
state_map = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA",
    "Colorado":"CO","Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA",
    "Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS",
    "Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA",
    "Michigan":"MI","Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT",
    "Nebraska":"NE","Nevada":"NV","New Hampshire":"NH","New Jersey":"NJ","New Mexico":"NM",
    "New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK",
    "Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC",
    "South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT",
    "Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY",
    "District of Columbia":"DC","Puerto Rico":"PR","Guam":"GU","Virgin Islands":"VI",
    "American Samoa":"AS","Northern Mariana Islands":"MP"
}

# --- Clean BLS wages (already loaded as bls_wages) ---
bls_wages["state_abbr"] = bls_wages["state"].map(state_map)

# --- Clean min wage (already loaded as min_wage) ---
min_wage["state_abbr"] = min_wage["state_name"].map(state_map)

# --- Compute medians from your dataset ---
df_state_medians = df_cleaned.groupby("state").agg(
    median_hourly_min=('hourly_min_final','median'),
    median_hourly_max=('hourly_max_final','median')
).reset_index()

# --- Merge everything using abbreviations ---
df_state = (
    df_state_medians
    # Merge in minimum wage
    .merge(min_wage[["state_abbr","state_min_wage"]],
           left_on="state", right_on="state_abbr", how="left")
    .drop(columns=["state_abbr"])   # drop immediately after first merge
    # Merge in BLS wages
    .merge(bls_wages[["state_abbr","hourly_median_preschool","hourly_median_kindergarten"]],
           left_on="state", right_on="state_abbr", how="left")
    .drop(columns=["state_abbr"])   # drop again after second merge
)


# --- Add national summary row ---
national_row = pd.DataFrame([{
    "state": "US",
    "median_hourly_min": df_cleaned['hourly_min_final'].median(),
    "median_hourly_max": df_cleaned['hourly_max_final'].median(),
    "state_min_wage": min_wage['state_min_wage'].median(),
    "hourly_median_preschool": bls_wages['hourly_median_preschool'].median(),
    "hourly_median_kindergarten": bls_wages['hourly_median_kindergarten'].median()
}])

df_final = pd.concat([national_row, df_state], ignore_index=True)

df_final.to_excel("median_hourly_wages_by_state.xlsx", index=False)


## Benefits

In [ ]:
import re
import pandas as pd
import json

# ------------------------------------------
# 1. Identify rows missing benefit text
# ------------------------------------------
missing_benefits_mask = df['benefits'].isna() | (df['benefits'].str.strip() == "")
df_missing_benefits = df.loc[missing_benefits_mask, ['jobid','description_text']].copy()

print("Jobs missing benefits info:", len(df_missing_benefits))

# ------------------------------------------
# 2. Regex function to detect standard benefits
# ------------------------------------------
benefit_keywords = [
    "health insurance", "dental insurance", "vision insurance",
    "401(k)", "retirement plan", "paid time off", "pto",
    "parental leave", "sick leave", "tuition reimbursement",
    "flexible schedule", "remote work", "life insurance",
    "disability insurance", "bonus", "professional development"
]

pattern = re.compile("|".join([re.escape(k) for k in benefit_keywords]), flags=re.IGNORECASE)

def regex_benefits(text):
    if pd.isna(text):
        return None
    found = pattern.findall(str(text))
    found_clean = sorted(set([f.lower() for f in found]))
    return ", ".join(found_clean) if found_clean else None

df_missing_benefits['regex_benefits'] = df_missing_benefits['description_text'].apply(regex_benefits)

print("Regex captured:", df_missing_benefits['regex_benefits'].notna().sum())
print("Still missing after regex:", df_missing_benefits['regex_benefits'].isna().sum())

# ------------------------------------------
# 3. GPT extraction for nuanced benefits
# ------------------------------------------
def extract_benefits_from_text(text, jobid=None):
    prompt = f"""
    Identify all **employee benefits** described in this job posting.
    Include items such as insurance, leave, bonuses, tuition assistance,
    schedule flexibility, and retirement plans. 
    Ignore generic language like "great benefits" or "competitive pay".
    List each benefit separated by commas. 
    If none are mentioned, return exactly "None".

    Text:
    {text}

    Respond strictly in JSON format:
    {{
        "benefits": "<comma-separated list of benefits or None>"
    }}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception:
        return None

# ------------------------------------------
# 4. Run GPT on a manageable sample
# ------------------------------------------
sample_rows = df_missing_benefits.head(100)  # safety limit; adjust upward as needed

gpt_results = []
for idx, row in sample_rows.iterrows():
    result = extract_benefits_from_text(row['description_text'], jobid=row['jobid'])
    gpt_results.append({"jobid": row['jobid'], "benefits_extracted": result})

df_gpt_benefits = pd.DataFrame(gpt_results)

# ------------------------------------------
# 5. Clean GPT JSON output
# ------------------------------------------
def clean_gpt_output(raw):
    if pd.isna(raw) or raw is None:
        return None
    cleaned = re.sub(r"^```json|```$", "", str(raw), flags=re.MULTILINE).strip()
    try:
        parsed = json.loads(cleaned)
        return parsed.get("benefits", None)
    except:
        return None

df_gpt_benefits['parsed_benefits'] = df_gpt_benefits['benefits_extracted'].apply(clean_gpt_output)

# ------------------------------------------
# 6. Merge results back
# ------------------------------------------
df_benefits_debug = sample_rows.merge(
    df_gpt_benefits[['jobid','parsed_benefits']],
    on='jobid', how='left'
)

df_benefits_debug['final_benefits'] = df_benefits_debug.apply(
    lambda r: r['regex_benefits'] if pd.notna(r['regex_benefits'])
              else (r['parsed_benefits'] if pd.notna(r['parsed_benefits']) else None),
    axis=1
)

# ------------------------------------------
# 7. Monitoring
# ------------------------------------------
print("GPT sample size:", len(sample_rows))
print("GPT parsed valid:", df_gpt_benefits['parsed_benefits'].notna().sum())
print("Final benefits captured:", df_benefits_debug['final_benefits'].notna().sum())

# Save to CSV for QC
df_benefits_debug.to_csv("benefits_extraction_sample.csv", index=False)
